<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/colbert_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q
!pip uninstall -y tensorflow -q
!pip install fuzzywuzzy[speedup] -q
!pip install networkx -q
!pip install pytorch-crf -q
!pip install pattern -q
import nltk
nltk.download('omw-1.4')

     |████████████████████████████████| 5.8 MB 5.4 MB/s 
     |████████████████████████████████| 182 kB 65.3 MB/s 
     |████████████████████████████████| 7.6 MB 20.3 MB/s 
     |████████████████████████████████| 174 kB 6.6 MB/s 
     |████████████████████████████████| 2.2 MB 66.9 MB/s 
     |████████████████████████████████| 22.2 MB 1.2 MB/s 
     |████████████████████████████████| 88 kB 8.9 MB/s 
     |████████████████████████████████| 81 kB 10.7 MB/s 
     |████████████████████████████████| 5.6 MB 53.5 MB/s 
     |████████████████████████████████| 5.6 MB 56.4 MB/s 
     |████████████████████████████████| 348 kB 71.6 MB/s 
     |████████████████████████████████| 100 kB 13.1 MB/s 
     |████████████████████████████████| 4.0 MB 60.2 MB/s 


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
!git clone https://github.com/meti-94/BERT-QA.git

Cloning into 'BERT-QA'...
remote: Enumerating objects: 616, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 616 (delta 37), reused 58 (delta 20), pack-reused 531
Receiving objects: 100% (616/616), 117.33 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (272/272), done.


In [3]:
%cd /content/BERT-QA/src
# model types : [MultiDepthNodeEdgeDetector, BertLSTMCRF, BertCNN, NodeEdgeDetector]
# cross validation: [True, False]
!python train.py NodeEdgeDetector False

/content/BERT-QA/src
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 139740646407760 on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:filelock:Lock 139740646407760 acquired on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 570
Downloading: 100% 570/570 [00:00<00:00, 745kB/s]
DEBUG:filelock:Attempting to release lock 139740646407760 on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:filelock:Lock 13974064

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
!cp /content/drive/MyDrive/reverb_wikipedia_tuples-1.1.txt /content/BERT-QA/data/

In [6]:
!pip install python-terrier -q
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git -q
import sys

COLAB='google.colab' in sys.modules

try:
  import faiss
  faiss.get_num_gpus()
except:
  if COLAB:
    print('Installing faiss-gpu from pip ')
    !pip install faiss-gpu==1.6.3 -q
  else:
    print('Installing faiss-gpu via Conda')
    !conda install -c pytorch faiss-gpu -q

import faiss
assert faiss.get_num_gpus() > 0


     |████████████████████████████████| 102 kB 5.4 MB/s 
     |████████████████████████████████| 1.5 MB 39.1 MB/s 
     |████████████████████████████████| 69 kB 3.9 MB/s 
     |████████████████████████████████| 311 kB 67.6 MB/s 
     |████████████████████████████████| 46 kB 4.6 MB/s 
     |████████████████████████████████| 286 kB 69.7 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 293 kB 68.8 MB/s 
     |████████████████████████████████| 1.2 MB 54.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 133 kB 77.1 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 16.5 MB 9.0 MB/s 
     |████████████████████████████████| 575 kB 80.8 MB/s 
     |████████████████████████████████| 147 kB 65.6 MB/s 
     |█████████

In [7]:
import pyterrier as pt
pt.init()
checkpoint="http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"
!rm -rf /content/colbertindex
import pyterrier_colbert.indexing
indexer = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint, "/content", "colbertindex", chunksize=3)
# indexer.index(pt.get_dataset("irds:vaswani").get_corpus_iter())

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [8]:
import pandas as pd
path = '/content/reverb_wikipedia_tuples-1.1.txt'
df = pd.read_csv(path, sep='\t', header=None)
reverb_columns_name = ['ExID', 'arg1', 'rel', 'arg2', 'narg1', 'nrel', 'narg2', 'csents', 'conf', 'urls']
df.columns = reverb_columns_name
df = df.dropna()
df = df.drop_duplicates()
df['normal_text'] = df.apply(lambda row:' '.join([row['narg1'], row['nrel'], row['narg2']]).strip(), axis=1)
df['text'] = df.apply(lambda row:' '.join([row['arg1'], row['rel'], row['arg2']]).strip(), axis=1)
df = df[['normal_text', 'text']]
df['docno'] = (df.index).astype(str)
df = df[:1000]

In [12]:
(indexer % 100).search('hi')

AttributeError: ignored

In [9]:
indexer.index(df[['text', 'docno']].to_dict(orient="records"))

[Dec 12, 14:44:27] [0] 		 #> Local args.bsize = 128
[Dec 12, 14:44:27] [0] 		 #> args.index_root = /content
[Dec 12, 14:44:27] [0] 		 #> self.possible_subset_sizes = [69905]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[Dec 12, 14:44:30] #> Loading model checkpoint.
[Dec 12, 14:44:30] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip


Downloading: "http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip" to /root/.cache/torch/hub/checkpoints/colbert.dnn.zip


  0%|          | 0.00/1.11G [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/torch/hub.py:651: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[Dec 12, 14:45:26] #> checkpoint['epoch'] = 0
[Dec 12, 14:45:26] #> checkpoint['batch'] = 44500




Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]



[Dec 12, 14:45:28] #> Note: Output directory /content already exists




[Dec 12, 14:45:28] #> Creating directory /content/colbertindex 


[Dec 12, 14:45:32] [0] 		 #> Completed batch #0 (starting at passage #0) 		Passages/min: 12.2k (overall),  12.3k (this encoding),  1981.6M (this saving)
[Dec 12, 14:45:33] [0] 		 [NOTE] Done with local share.
[Dec 12, 14:45:33] [0] 		 #> Saved batch #0 to /content/colbertindex/0.pt 		 Saving Throughput = 4.3M passages per minute.

[Dec 12, 14:45:33] [0] 		 #> Joining saver thread.
#> num_embeddings = 11543
[Dec 12, 14:45:33] #> Starting..
[Dec 12, 14:45:33] #> Processing slice #1 of 1 (range 0..1).
[Dec 12, 14:45:33] #> Will write to /content/colbertindex/ivfpq.100.faiss.
[Dec 12, 14:45:33] #> Loading /content/colbertindex/0.sample ...
#> Sample has shape (577, 128)
[Dec 12, 14:45:33] Preparing resources for 1 GPUs.
[Dec 12, 14:45:33] #> Training with the vectors...
[Dec 12, 14:45:33] #> Training now (using 1 GPUs)...
107.00880026817322
0.79742145

In [10]:
pyterrier_colbert_factory = indexer.ranking_factory()
colbert_e2e = pyterrier_colbert_factory.end_to_end()
 

[Dec 12, 14:47:57] #> Loading the FAISS index from /content/colbertindex/ivfpq.100.faiss ..
[Dec 12, 14:47:57] #> Building the emb2pid mapping..
[Dec 12, 14:47:57] len(self.emb2pid) = 11543
Loading reranking index, memtype=mem


Loading index shards to memory:   0%|          | 0/1 [00:00<?, ?shard/s]

In [18]:
(colbert_e2e % 10).search("chemical reactions").sort_values(by='score', ascending=False)

,qid,query,docid,query_toks,query_embs,score,docno,rank
71,1,chemical reactions,84,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",13.359938,84,0
885,1,chemical reactions,942,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",12.996468,942,1
555,1,chemical reactions,604,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",12.604588,604,2
716,1,chemical reactions,768,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",12.554459,768,3
263,1,chemical reactions,298,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",11.858632,298,4
469,1,chemical reactions,516,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",11.562166,516,5
397,1,chemical reactions,439,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",11.046332,439,6
238,1,chemical reactions,272,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",10.871019,272,7
296,1,chemical reactions,334,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",10.789043,334,8
471,1,chemical reactions,518,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0681), tensor(-0.0084), tensor(0.11...",10.763349,518,9


In [ ]:
from train import *
from graph import *
from utils import get_hit
import sys
import re

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bert = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
node_edge_detector = NodeEdgeDetector(bert, tokenizer, dropout=torch.tensor(0.5))
optimizer = AdamW
kw = {'lr':0.0002, 'weight_decay':0.1}
tl = TrainingLoop(node_edge_detector, optimizer, True, **kw)
loss = mse_loss
tl.load()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

In [ ]:
test_df = pd.read_excel('/content/BERT-QA/data/test.xlsx')
actual = test_df['Reverb_no'].to_list()
system_results = []
for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
  # node, edge = tl.readable_predict(device, _input=row['Question'], print_result=False)
  # node = ' '.join(node); edge = ' '.join(edge)
  # node = node.replace(' ##', ''); edge = edge.replace(' ##', '')
  # if index%1000 == 0:
  #   print(f"Question Number {index+1}: ", row['Question'].lower().split())
  #   print(f'Node: {node}, Edge: {edge}')
  # q = ' '.join([node, edge])
  q = row['Question']
  q = re.sub(r'[^\w]', ' ', q)
  temp = (colbert_e2e % 10).search(q)['docno'].to_list()
  temp = [[int(item)] for item in temp]
  system_results.append(temp)
  
print(get_hit(actual, system_results))


100%|██████████| 5003/5003 [40:12<00:00,  2.07it/s]

(0.7955226863881671, 0.932240655606636, 0.9520287827303617, 0.9700179892064761, 0.9700179892064761)


In [ ]:
!rm -rf /content/colbertindex
!python evaluation.py NN

Streaming output truncated to the last 5000 lines.
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:
meth: wait
  sig: (JI)V
  Public True
  Private False
  Protected False
  Static False
  Final True
  Synchronized False
  Volatile False
  Transient False
  Native False
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:
meth: wait
  sig: ()V
  Public True
  Private False
  Protected False
  Static False
  Final True
  Synchronized False
  Volatile False
  Transient False
  Native False
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:method selected 3 multiple signatures of [('(J)V', False, False), ('(JI)V', False, False), ('()V', False, False)]
DEBUG:jnius.reflect:
meth: equals
  sig: (Ljava/lang/Object;)Z
  Public True
  Private False
  Protected False
  Static False
  Final False
  Synchronized False
  Volatile False
  Transient False
  Native False
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflec

In [34]:
!rm -rf /content/colbertindex
!python graph.py

Streaming output truncated to the last 5000 lines.
  Static False
  Final False
  Synchronized False
  Volatile False
  Transient False
  Native False
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:
meth: initialise
  sig: (Lorg/terrier/querying/SearchRequest;)V
  Public False
  Private False
  Protected True
  Static False
  Final False
  Synchronized False
  Volatile False
  Transient False
  Native False
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:
meth: getModel
  sig: (Ljava/lang/String;I)Lorg/terrier/matching/models/WeightingModel;
  Public False
  Private False
  Protected False
  Static False
  Final False
  Synchronized False
  Volatile False
  Transient False
  Native False
  Interface False
  Abstract False
  Strict False

DEBUG:jnius.reflect:autoclass(java.lang.reflect.Proxy) intf False hierarchy is [(<Class: java.lang.Object at 0x7f8ec74f8400>, 1), (<Interface: java.io.Serializable at 0x7f8ec751a3b0>, 1), (<Class: java.lan